# Informações do ambiente

In [ ]:
import sys
import os
import platform
import ipykernel

print("OK kernel")
print("Python exe:", sys.executable) # Qual interpretador está sendo usado
print("Python ver:", sys.version.split()[0]) # versão do Python
print("ipykernel:", ipykernel.__version__) # versão do Kernel
print("CWD:", os.getcwd()) # diretório de trabalho atual
print("SO:", platform.platform()) # sistema operacional

OK kernel
Python exe: c:\Users\fabio\Documents\CIência de Dados\15. Estudos VS\20260228 - BeautifulSoup\.venv\Scripts\python.exe
Python ver: 3.11.9
ipykernel: 7.2.0
CWD: c:\Users\fabio\Documents\CIência de Dados\15. Estudos VS\20260228 - BeautifulSoup
SO: Windows-10-10.0.26200-SP0


# Teste de requisição
- Faz uma requisição HTTP simples (requests.get)
- Verifica se houve erro (raise_for_status)
- Cria um objeto BeautifulSoup
- Extrai: Título da página e Primeiros 5 livros (título e preço)

In [3]:
import requests
from bs4 import BeautifulSoup

url = "http://books.toscrape.com/"

resp = requests.get(url, timeout=15)
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "lxml")

print("Título da página:", soup.title.get_text(strip=True))

books = []
for art in soup.select("article.product_pod")[:5]:
    title = art.h3.a["title"]
    price = art.select_one(".price_color").get_text(strip=True)
    books.append((title, price))

print("Amostra de livros:", books)

Título da página: All products | Books to Scrape - Sandbox
Amostra de livros: [('A Light in the Attic', 'Â£51.77'), ('Tipping the Velvet', 'Â£53.74'), ('Soumission', 'Â£50.10'), ('Sharp Objects', 'Â£47.82'), ('Sapiens: A Brief History of Humankind', 'Â£54.23')]


# Sessão com retry
- Header User-Agent customizado: identifica quem está fazendo a requisição
- Estratégia de retry automática
- Backoff exponencial: tempo de espera crescente, reduz risco de bloqueio
- Tratamento de códigos de erro específicos: retry somente para erros temporários

In [5]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def make_session():
    s = requests.Session()
    
    s.headers.update({
        "User-Agent": "raspatela/1.0 (+Fabio)"
    })
    
    retry = Retry(
        total=5,  # até 5 tentativas
        backoff_factor=0.7,  # exponencial: 0.7, 1.4, 2.8...
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD"],
        raise_on_status=False,
    )
    
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    
    return s

session = make_session()

# Outra abordagem
## Função de fetch


In [2]:
def fetch(url, timeout=15):
    r = session.get(url, timeout=timeout)
    
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code} em {url}")
    
    return r.text

## Função de parsing

In [3]:
from bs4 import BeautifulSoup

def parse_books(html):
    soup = BeautifulSoup(html, "lxml")
    out = []

    for art in soup.select("article.product_pod"):
        title = art.h3.a["title"].strip()
        price = art.select_one(".price_color").get_text(strip=True)
        stock = art.select_one(".availability").get_text(strip=True)

        out.append({
            "titulo": title,
            "preco": price,
            "estoque": stock
        })

    return out

## Rodar

In [6]:
import pandas as pd

url = "http://books.toscrape.com/"

html = fetch(url)
books = parse_books(html)

print("Total de livros encontrados:", len(books))
print("Amostra:", books[:5])

df = pd.DataFrame(books)

print("Dimensão:", df.shape)
df.head()

Total de livros encontrados: 20
Amostra: [{'titulo': 'A Light in the Attic', 'preco': 'Â£51.77', 'estoque': 'In stock'}, {'titulo': 'Tipping the Velvet', 'preco': 'Â£53.74', 'estoque': 'In stock'}, {'titulo': 'Soumission', 'preco': 'Â£50.10', 'estoque': 'In stock'}, {'titulo': 'Sharp Objects', 'preco': 'Â£47.82', 'estoque': 'In stock'}, {'titulo': 'Sapiens: A Brief History of Humankind', 'preco': 'Â£54.23', 'estoque': 'In stock'}]
Dimensão: (20, 3)


,titulo,preco,estoque
0,A Light in the Attic,Â£51.77,In stock
1,Tipping the Velvet,Â£53.74,In stock
2,Soumission,Â£50.10,In stock
3,Sharp Objects,Â£47.82,In stock
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock
